# Master Prepare Notebook

Deze notebook wordt gebruikt om alle data uit de datasets in te laden en verder te verwerken, zodat deze klaar staat om modellen te trainen.

In [1]:
# Load public modules.
import os, sys
from sklearn.pipeline import Pipeline
from pathlib import Path
import pickle
import time

# Get the home dir and username.
HOME = Path.home()
USERNAME = os.path.basename(HOME)

# Set codebase path for old VAO.
CODEBASE_PATH_OLD = os.path.join(HOME, 'Documents/woonfraude/codebase/')
sys.path.insert(1, CODEBASE_PATH_OLD)
                
# Set codebase path for new VAO.
CODEBASE_PATH_NEW = os.path.join('/data', USERNAME, 'Documents/woonfraude/codebase/')
sys.path.insert(1, CODEBASE_PATH_NEW)

# Import own modules.
from datasets import *
from clean import *
from extract_features import *

# Load all datasets in memory

In [2]:
# Initiate dataset objects.
zakenDataset = ZakenDataset()
stadiaDataset = StadiaDataset()

In [3]:
# Load data from files.
zakenDataset.load('final')
stadiaDataset.load('download_ids')

Version 'final' of dataset 'zaken' loaded!
Version 'download_ids' of dataset 'stadia' loaded!


In [4]:
# Assign dataframes to more simple variable names for easy access.
zaken = zakenDataset.data
stadia = stadiaDataset.data

In [5]:
zaken_totaal = pd.merge(zaken, stadia, how='left', on=['adres_id','wvs_nr'])

In [6]:
#lijst van alle Buitendienst stadia
bd_lijst = ['2de controle','3de controle','onderzoek buitendienst','avondronde','weekend buitendienstonderzoek','buitendienstonderzoek']
print  (bd_lijst)

['2de controle', '3de controle', 'onderzoek buitendienst', 'avondronde', 'weekend buitendienstonderzoek', 'buitendienstonderzoek']


In [7]:
indicatie_buitendienst = []

#create an indication buitendienst
for value in zaken_totaal['sta_oms']:
    if str(value).lower() in ['2de controle','3de controle','onderzoek buitendienst','avondronde','weekend buitendienstonderzoek','buitendienstonderzoek']:
        indicatie_buitendienst.append(1)
    else:
        indicatie_buitendienst.append(0)

In [8]:
sum(indicatie_buitendienst)

18992

In [9]:
# add indicatie buitendienst to zaken_totaal
zaken_totaal['indicatie_buitendienst'] = indicatie_buitendienst

In [10]:
def BD_belopen(df):

    if (df['indicatie_buitendienst'] == 1 and str(df['einddatum_x']) != ''  and str(df['einddatum_y']) != ''):
        return 1
    else:
        return 0


zaken_totaal['indicatie_buitendienst_belopen'] = zaken_totaal.apply(BD_belopen, axis = 1)

In [11]:
BD_opgevoerd = sum(zaken_totaal['indicatie_buitendienst'])
BD_belopen = sum(zaken_totaal['indicatie_buitendienst_belopen'])

print(BD_opgevoerd)
print(BD_belopen)

18992
18992


- is gelijk omdat er alleen afgesloten zaken in de data zitten

In [12]:
zaken_totaal['categorie'].head()

BD_per_project = zaken_totaal.groupby(['categorie'])['indicatie_buitendienst_belopen'].sum()

BD_per_project.sort_values(ascending=False)


categorie
illegale hotels                             8821
leegstand                                   3182
woningdelen                                 2123
laa                                         2122
onrechtmatige bewoning                      1152
missing                                      890
illegale onderhuur                           275
buurtprojecten                               260
inschrijvingsfraude                          109
in gebruik zonder huisvestingsvergunning      29
prostitutie                                   26
drugs                                          3
uitkeringsfraude                               0
Name: indicatie_buitendienst_belopen, dtype: int64

- is de laatste versie geschikt voor deze berekening?

In [13]:
#assigns handhavingsstadia to variable
hb_lijst = ['opleggen boete','last onder dwangsom','opleggen boete doorverhuurder','meldplicht beschikking boete','last onder bestuursdwang','bew(huid) last onder dwangsom','eig. last onder dwangsom','bestuurlijke boete zaak','sluiting (ivm brandveiligheid)','opleggen boete eigenaar']


In [14]:
indicatie_handhaving = []

#create and indication buitendienst
for value in zaken_totaal['sta_oms']:
    if str(value).lower() in hb_lijst:
        indicatie_handhaving.append(1)
    else:
        indicatie_handhaving.append(0)

In [15]:
# add indicatie buitendienst to zaken_totaal
zaken_totaal['indicatie_handhaving'] = indicatie_handhaving

In [16]:
zaken_totaal['categorie'].head()

HB_per_project = zaken_totaal.groupby(['categorie'])['indicatie_handhaving'].sum()

HB_per_project.sort_values(ascending=False)

categorie
illegale hotels                             1709
woningdelen                                  533
missing                                      113
onrechtmatige bewoning                        90
leegstand                                     19
buurtprojecten                                13
inschrijvingsfraude                            4
in gebruik zonder huisvestingsvergunning       4
prostitutie                                    3
drugs                                          3
illegale onderhuur                             2
uitkeringsfraude                               0
laa                                            0
Name: indicatie_handhaving, dtype: int64

In [17]:
#makes an indication of the amount of handhavingsbesluiten related to the amount of buitendienstonderzoeken

HB_per_BD = HB_per_project/BD_per_project
HB_per_BD.sort_values(ascending=False)

categorie
drugs                                       1.000000
woningdelen                                 0.251060
illegale hotels                             0.193742
in gebruik zonder huisvestingsvergunning    0.137931
missing                                     0.126966
prostitutie                                 0.115385
onrechtmatige bewoning                      0.078125
buurtprojecten                              0.050000
inschrijvingsfraude                         0.036697
illegale onderhuur                          0.007273
leegstand                                   0.005971
laa                                         0.000000
uitkeringsfraude                                 NaN
dtype: float64

# Vakantieverhuur

In [18]:
vakantieverhuur = zaken_totaal.loc[zaken_totaal['categorie'].isin(['illegale hotels'])]

In [19]:
fraude_count = 0

for boolean in vakantieverhuur[['zaak_id_x','woonfraude']].drop_duplicates()['woonfraude']:
    if boolean == True:
        fraude_count +=1
        
print('The percentage of fraud withing vakantieverhuur is: '+str(round(fraude_count/len(vakantieverhuur[['zaak_id_x','woonfraude']].drop_duplicates()['woonfraude'])*100,1))+'%')

The percentage of fraud withing vakantieverhuur is: 37.5%


In [20]:
#counts the cases per project
zaken_per_project_VV = vakantieverhuur.groupby(['beh_oms'])['zaak_id_x'].nunique()
zaken_per_project_VV.sort_values(ascending=False)

beh_oms
hotline                                 2579
zkl illegaal hotel/shortstay            1563
zebra                                   1449
safari                                   558
safari 2015                              328
woonschip (woonboot)                     132
safari 2                                 104
vakantie juli 2014                        88
burgwallenproject oudezijde               80
nz-buurt                                  76
haarlemmerbuurt                           74
safari 3                                  73
wolga                                     69
digitaal toezicht safari                  69
digitaal toezicht twitter                 65
project jordaan                           63
project lobith                            56
zkl doorzon illegaal hotel/shortstay      52
han hercontrole                           37
digitaal toezicht zebra                   13
mystery guest                             10
zkl vakantie verhuur overlast             10
va

-Sommige projecten zijn niet van de grond gekomen (minder dan 30 zaken)

In [21]:
#counts the 'Buitendienst onderzoeken' per project
BD_per_project_VV = vakantieverhuur.groupby(['beh_oms'])['indicatie_buitendienst'].sum()
BD_per_project_VV.sort_values(ascending=False)

beh_oms
hotline                                 4394
zebra                                   1799
safari                                   750
safari 2015                              442
woonschip (woonboot)                     212
safari 2                                 125
burgwallenproject oudezijde              118
project jordaan                          116
digitaal toezicht twitter                112
digitaal toezicht safari                 111
haarlemmerbuurt                          109
vakantie juli 2014                        99
nz-buurt                                  97
safari 3                                  81
wolga                                     72
project lobith                            62
han hercontrole                           39
digitaal toezicht zebra                   29
zkl illegaal hotel/shortstay              24
mystery guest                             12
vakantieverhuur 60 dagen  project          5
project molensteeg                         5
si

In [22]:
#counts the 'Handhavingsbesluiten' per project
HB_per_project_VV = vakantieverhuur.groupby(['beh_oms'])['indicatie_handhaving'].sum()
HB_per_project_VV.sort_values(ascending=False)



beh_oms
hotline                                 805
safari                                  267
zebra                                   260
safari 2015                             138
woonschip (woonboot)                     56
digitaal toezicht safari                 33
burgwallenproject oudezijde              29
safari 2                                 22
nz-buurt                                 19
haarlemmerbuurt                          18
project jordaan                          12
wolga                                    11
digitaal toezicht twitter                10
mystery guest                             9
han hercontrole                           5
project lobith                            4
safari 3                                  3
zkl illegaal hotel/shortstay              3
vakantie juli 2014                        2
sint annekwartier                         1
project molensteeg                        1
digitaal toezicht zebra                   1
vakantieverhuur 60 dagen

-Alleen Hotline, Safari en Zebra hebben echt wat opgeleverd

In [23]:
#creates a series which has a bool for more than 2 buitendienst onderzoeken and than turns it into a dataframe
BD_per_zaak_VV = vakantieverhuur.groupby(['zaak_id_x'])['indicatie_buitendienst'].sum()>2
BD_per_zaak_VV.sort_values(ascending=False)

BD_per_zaak_VV


meer_dan_twee = pd.DataFrame([BD_per_zaak_VV])

In [24]:
meer_dan_twee = meer_dan_twee.transpose()

meer_dan_twee['zaak_id'] = meer_dan_twee.index

In [25]:
meer_dan_twee.head()

,indicatie_buitendienst,zaak_id
zaak_id_x,,
100025_1,False,100025_1
100026_1,False,100026_1
100027_1,False,100027_1
100305_2,False,100305_2
100322_1,False,100322_1


In [26]:
meer_dan_twee = meer_dan_twee[['indicatie_buitendienst','zaak_id']]

In [27]:
#makes a list of zaak_ids with less than 3 buitendienst onderzoeken
zaak_ids_minderdan3 = list(meer_dan_twee.loc[meer_dan_twee['indicatie_buitendienst'] == False]['zaak_id'])


In [60]:
vakantieverhuur = zaken_totaal.loc[zaken_totaal['categorie'].isin(['illegale hotels'])]
vakantieverhuur.head()

,adres_id,afg_code_afs_x,afg_code_beh,afs_code_x,afs_oms_x,bedrag_huur,begindatum_x,beh_code,beh_oms,categorie,...,date_created,date_modified,wzs_id_y,stadia_id,wzs_update_datumtijd_y,zaak_id_y,stadium_id,indicatie_buitendienst,indicatie_buitendienst_belopen,indicatie_handhaving
0,249960.0,vtbd,vtbd,pgwg,zl geen woonfraude,447.0,2015-07-27,zkl_hos,zkl illegaal hotel/shortstay,illegale hotels,...,2015-07-27,2016-02-25,332304164.0,249960_2_2,2019-08-28 13:09:32.065550,249960_2,249960_2_2,0,0,0
1,249960.0,vtbd,vtbd,pgwg,zl geen woonfraude,447.0,2015-07-27,zkl_hos,zkl illegaal hotel/shortstay,illegale hotels,...,2015-07-27,None,332357839.0,249960_2_1,2019-08-28 13:09:32.065550,249960_2,249960_2_1,0,0,0
17,375774.0,vtbd,vtbd,pwibg,zl woning is beschikbaar gekomen,334.0,2015-11-02,zkl_hos,zkl illegaal hotel/shortstay,illegale hotels,...,2015-11-02,2016-01-05,332251552.0,375774_2_1,2019-08-28 13:09:32.065550,375774_2,375774_2_1,0,0,0
18,213839.0,vtbd,vtbd,pwibg,zl woning is beschikbaar gekomen,786.0,2015-03-02,zkl_hos,zkl illegaal hotel/shortstay,illegale hotels,...,2016-03-03,None,332237685.0,213839_2_2,2019-08-28 13:09:32.065550,213839_2,213839_2_2,0,0,0
19,213839.0,vtbd,vtbd,pwibg,zl woning is beschikbaar gekomen,786.0,2015-03-02,zkl_hos,zkl illegaal hotel/shortstay,illegale hotels,...,2016-03-03,2016-05-31,332264424.0,213839_2_1,2019-08-28 13:09:32.065550,213839_2,213839_2_1,0,0,0


In [61]:
vakantieverhuur = vakantieverhuur.loc[vakantieverhuur['zaak_id_y'].isin(zaak_ids_minderdan3)]

In [62]:
vakantieverhuur.head()

,adres_id,afg_code_afs_x,afg_code_beh,afs_code_x,afs_oms_x,bedrag_huur,begindatum_x,beh_code,beh_oms,categorie,...,date_created,date_modified,wzs_id_y,stadia_id,wzs_update_datumtijd_y,zaak_id_y,stadium_id,indicatie_buitendienst,indicatie_buitendienst_belopen,indicatie_handhaving
0,249960.0,vtbd,vtbd,pgwg,zl geen woonfraude,447.0,2015-07-27,zkl_hos,zkl illegaal hotel/shortstay,illegale hotels,...,2015-07-27,2016-02-25,332304164.0,249960_2_2,2019-08-28 13:09:32.065550,249960_2,249960_2_2,0,0,0
1,249960.0,vtbd,vtbd,pgwg,zl geen woonfraude,447.0,2015-07-27,zkl_hos,zkl illegaal hotel/shortstay,illegale hotels,...,2015-07-27,None,332357839.0,249960_2_1,2019-08-28 13:09:32.065550,249960_2,249960_2_1,0,0,0
17,375774.0,vtbd,vtbd,pwibg,zl woning is beschikbaar gekomen,334.0,2015-11-02,zkl_hos,zkl illegaal hotel/shortstay,illegale hotels,...,2015-11-02,2016-01-05,332251552.0,375774_2_1,2019-08-28 13:09:32.065550,375774_2,375774_2_1,0,0,0
18,213839.0,vtbd,vtbd,pwibg,zl woning is beschikbaar gekomen,786.0,2015-03-02,zkl_hos,zkl illegaal hotel/shortstay,illegale hotels,...,2016-03-03,None,332237685.0,213839_2_2,2019-08-28 13:09:32.065550,213839_2,213839_2_2,0,0,0
19,213839.0,vtbd,vtbd,pwibg,zl woning is beschikbaar gekomen,786.0,2015-03-02,zkl_hos,zkl illegaal hotel/shortstay,illegale hotels,...,2016-03-03,2016-05-31,332264424.0,213839_2_1,2019-08-28 13:09:32.065550,213839_2,213839_2_1,0,0,0


In [63]:
fraude_count = 0

for boolean in vakantieverhuur[['zaak_id_x','woonfraude']].drop_duplicates()['woonfraude']:
    if boolean == True:
        fraude_count +=1
        
print(fraude_count/len(vakantieverhuur[['zaak_id_x','woonfraude']].drop_duplicates()['woonfraude']))


0.3853003320253547


In [64]:
#checks the percentage of fraud within all cases
fraude_count = 0

for boolean in zaken_totaal[['zaak_id_x','woonfraude']].drop_duplicates()['woonfraude']:
    if boolean == True:
        fraude_count +=1
        
print(fraude_count/len(zaken_totaal[['zaak_id_x','woonfraude']].drop_duplicates()))

0.5331516802906449


In [68]:
# Checks the percentage of fraud within every project
for project in set(list(vakantieverhuur['beh_oms'])):
    table = vakantieverhuur.loc[vakantieverhuur['beh_oms'].isin([project])]
    table = table.drop_duplicates(subset=['zaak_id_x','woonfraude'])
    woonfraude = table['woonfraude']  # Remove duplicates and select woonfraude column.

    # Print results.
    count = sum(woonfraude)
    numer_of_cases = len(woonfraude)
    fraud_percentage = round(count/numer_of_cases*100,1)
    print(f'We have a fraud ratio of {fraud_percentage}% with {numer_of_cases} zaken for project {project}.')

We have a fraud ratio of 88.9% with 72 zaken for project safari 3.
We have a fraud ratio of 48.6% with 107 zaken for project woonschip (woonboot).
We have a fraud ratio of 62.1% with 512 zaken for project safari.
We have a fraud ratio of 25.0% with 52 zaken for project digitaal toezicht twitter.
We have a fraud ratio of 40.7% with 59 zaken for project digitaal toezicht safari.
We have a fraud ratio of 57.1% with 7 zaken for project digitaal toezicht zebra.
We have a fraud ratio of 0.0% with 1 zaken for project sint annekwartier.
We have a fraud ratio of 93.1% with 101 zaken for project safari 2.
We have a fraud ratio of 29.4% with 1890 zaken for project hotline.
We have a fraud ratio of 62.1% with 66 zaken for project haarlemmerbuurt.
We have a fraud ratio of 53.6% with 1388 zaken for project zebra.
We have a fraud ratio of 20.0% with 10 zaken for project zkl vakantie verhuur overlast.
We have a fraud ratio of 66.7% with 75 zaken for project nz-buurt.
We have a fraud ratio of 36.9% wit

- lijkt redelijk hoog allemaal?

In [34]:
#checks if there are no double labels per zaak_id
df = vakantieverhuur.groupby('zaak_id_x')['woonfraude'].nunique()
df.sort_values(ascending=False)

zaak_id_x
99922_2      1
192640_2     1
192594_1     1
192598_1     1
192599_2     1
192602_2     1
192602_3     1
192603_2     1
192603_3     1
192604_3     1
192604_4     1
192639_1     1
192640_3     1
192493_1     1
192641_6     1
1926604_1    1
192680_2     1
1927206_1    1
1927206_4    1
192802_2     1
192805_5     1
192806_1     1
1928279_1    1
1928281_2    1
1925173_1    1
192491_1     1
1928281_6    1
1922834_2    1
1922186_1    1
1922186_2    1
            ..
276837_5     1
276837_6     1
276838_2     1
276838_3     1
276736_1     1
276718_6     1
276718_5     1
276718_2     1
27569_2      1
27569_3      1
275815_2     1
27583_2      1
276492_1     1
276502_1     1
276503_1     1
276534_3     1
276537_2     1
276539_1     1
276540_7     1
276588_1     1
276605_5     1
276606_2     1
276616_1     1
276693_1     1
276716_2     1
276716_3     1
276716_4     1
276717_1     1
276717_3     1
100025_1     1
Name: woonfraude, Length: 6626, dtype: int64

# Corporatiewoningen

In [35]:
for c in zaken_totaal.columns:
    print(c)

adres_id
afg_code_afs_x
afg_code_beh
afs_code_x
afs_oms_x
bedrag_huur
begindatum_x
beh_code
beh_oms
categorie
eigenaar
einddatum_x
kamer_aantal
mededelingen
nuttig_woonoppervlak
vloeroppervlak_totaal
wvs_nr
wzs_id_x
wzs_update_datumtijd_x
zaak_id_x
woonfraude
pvh_omschr
sbw_omschr
sbv_omschr
wzs_lon
wzs_lat
sttnaam
hsnr
hsltr
toev
sdl_naam
inwnrs
leegstand
ads_id
wng_id
postcode_x
id_nummeraanduiding
landelijk_id_nummeraanduiding
huisnummer_nummeraanduiding
huisletter_nummeraanduiding
huisnummer_toevoeging_nummeraanduiding
type
adres_nummer
hoofdadres
bron_id_nummeraanduiding
ligplaats_id
openbare_ruimte_id
standplaats_id
status_id_nummeraanduiding
verblijfsobject_id
_geom
bron_id_ligplaats
status_id_ligplaats
geometrie_standplaats
bron_id_standplaats
status_id_standplaats
oppervlakte
bouwlaag_toegang
status_coordinaat_code
bouwlagen
type_woonobject_omschrijving
aantal_kamers
geometrie_verblijfsobject
_huisletter_verblijfsobject
eigendomsverhouding_id
financieringswijze_id
gebruik_id
l

In [72]:
#creates dataframe with woco cases
wocos = vakantieverhuur = zaken_totaal.loc[zaken_totaal['eigenaar']!='particulier']
print(len(wocos))
print(len(zaken_totaal))

10267
89278


In [37]:
BD_per_categorie_wocos = wocos.groupby(['categorie'])['indicatie_buitendienst_belopen'].sum()

BD_per_categorie_wocos.sort_values(ascending=False)

categorie
laa                                         617
illegale hotels                             435
leegstand                                   304
woningdelen                                 269
buurtprojecten                              210
missing                                     186
illegale onderhuur                          116
inschrijvingsfraude                          81
onrechtmatige bewoning                       22
prostitutie                                   5
in gebruik zonder huisvestingsvergunning      1
drugs                                         0
Name: indicatie_buitendienst_belopen, dtype: int64

In [38]:
BD_per_project_wocos = wocos.groupby(['beh_oms'])['indicatie_buitendienst_belopen'].sum()

BD_per_project_wocos.sort_values(ascending=False)

beh_oms
hotline                                       377
laa_samenwoners                               150
laa_overbewoning                              143
laa                                           127
laa_veelverhuizers                            111
laa_status_vow                                108
leegstand administratief                      102
laa_meldingen                                  87
harmocorp                                      80
laa_doorgangsadres                             78
zoekl.project b83                              72
datos - melding bi                             50
zoekl.project b84                              47
corpo-rico                                     42
leegstand feitelijk                            39
kleiburg                                       36
hoogoord                                       33
woningdelen corporaties                        31
zoeklicht flits                                28
zoekl.project b82                         

In [39]:
HB_per_categorie_wocos = wocos.groupby(['categorie'])['indicatie_handhaving'].sum()

HB_per_categorie_wocos.sort_values(ascending=False)

categorie
woningdelen                                 45
illegale hotels                             35
missing                                      8
prostitutie                                  1
leegstand                                    1
in gebruik zonder huisvestingsvergunning     1
onrechtmatige bewoning                       0
laa                                          0
inschrijvingsfraude                          0
illegale onderhuur                           0
drugs                                        0
buurtprojecten                               0
Name: indicatie_handhaving, dtype: int64

In [40]:
HB_per_project_wocos = wocos.groupby(['beh_oms'])['indicatie_handhaving'].sum()

HB_per_project_wocos.sort_values(ascending=False)

beh_oms
hotline                                       34
kleiburg                                      26
corpo-rico                                     4
hoogoord                                       3
tatra                                          3
tatra 2                                        2
spookburgers                                   2
woningdelen                                    2
woningdelen corporaties                        2
woningdelen laa                                2
woningkwaliteit fase ii vens                   2
sahara veel adv                                2
spookburgers mobiel banditisme                 1
spookburgers 1                                 1
spookburgers illegale prostitutie woningen     1
laa                                            1
nz-buurt                                       1
prostitutie                                    1
bsn 2                                          1
laa_status_vow                                 0
laa_veelverh

In [71]:
for project in set(list(wocos['beh_oms'])):
    tabel = wocos.loc[vakantieverhuur['beh_oms'].isin([project])]                                
    count = 0                        
    for boolean in tabel[['zaak_id_x','woonfraude']].drop_duplicates()['woonfraude']:
        if boolean == True:
            count +=1
        
    print('the fraud ratio of '+project+' =             '+str(round(count/len(tabel[['zaak_id_x','woonfraude']].drop_duplicates()['woonfraude'])*100,1))+'% en het aantal')

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match

In [73]:
# Checks the percentage of fraud within every project
for project in set(list(wocos['categorie'])):
    table = wocos.loc[vakantieverhuur['categorie'].isin([project])]
    table = table.drop_duplicates(subset=['zaak_id_x','woonfraude'])
    woonfraude = table['woonfraude']  # Remove duplicates and select woonfraude column.

    # Print results.
    count = sum(woonfraude)
    numer_of_cases = len(woonfraude)
    fraud_percentage = round(count/numer_of_cases*100,1)
    print(f'We have a fraud ratio of {fraud_percentage}% with {numer_of_cases} zaken for project {project}.')

We have a fraud ratio of 85.6% with 702 zaken for project drugs.
We have a fraud ratio of 77.6% with 76 zaken for project inschrijvingsfraude.
We have a fraud ratio of 41.0% with 222 zaken for project buurtprojecten.
We have a fraud ratio of 51.7% with 296 zaken for project woningdelen.
We have a fraud ratio of 40.1% with 137 zaken for project missing.
We have a fraud ratio of 0.0% with 1 zaken for project in gebruik zonder huisvestingsvergunning.
We have a fraud ratio of 55.0% with 271 zaken for project illegale onderhuur.
We have a fraud ratio of 46.0% with 250 zaken for project leegstand.
We have a fraud ratio of 51.4% with 319 zaken for project laa.
We have a fraud ratio of 23.6% with 501 zaken for project illegale hotels.
We have a fraud ratio of 66.7% with 30 zaken for project onrechtmatige bewoning.
We have a fraud ratio of 64.2% with 137 zaken for project prostitutie.


In [74]:
# Checks the percentage of fraud within every project
for project in set(list(wocos['beh_oms'])):
    table = wocos.loc[vakantieverhuur['beh_oms'].isin([project])]
    table = table.drop_duplicates(subset=['zaak_id_x','woonfraude'])
    woonfraude = table['woonfraude']  # Remove duplicates and select woonfraude column.

    # Print results.
    count = sum(woonfraude)
    numer_of_cases = len(woonfraude)
    fraud_percentage = round(count/numer_of_cases*100,1)
    print(f'We have a fraud ratio of {fraud_percentage}% with {numer_of_cases} zaken for project {project}.')

We have a fraud ratio of 100.0% with 1 zaken for project safari 3.
We have a fraud ratio of 100.0% with 5 zaken for project tatra 2.
We have a fraud ratio of 52.6% with 19 zaken for project hoogoord.
We have a fraud ratio of 29.4% with 17 zaken for project spookburgers 2.
We have a fraud ratio of 100.0% with 13 zaken for project woningdelen laa.
We have a fraud ratio of 53.8% with 26 zaken for project zoeklicht flits.
We have a fraud ratio of 17.6% with 51 zaken for project laa_meldingen.
We have a fraud ratio of 0.0% with 3 zaken for project zoekl.project b68.
We have a fraud ratio of 100.0% with 1 zaken for project meldnov2015.
We have a fraud ratio of 59.0% with 61 zaken for project laa_veelverhuizers.
We have a fraud ratio of 24.5% with 196 zaken for project zkl illegaal hotel/shortstay.
We have a fraud ratio of 87.5% with 8 zaken for project safari 2015.
We have a fraud ratio of 96.0% with 25 zaken for project zoekl.project b80.
We have a fraud ratio of 0.0% with 3 zaken for proje

- in een tabel wegschrijven zodat ik daarna kan sorteren